In [28]:
pip install PDFplumber

  Using cached Wand-0.6.7-py2.py3-none-any.whl (139 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [33]:
import pandas as pd
import pdfplumber
import re

In [43]:
pdf = pdfplumber.open(".\\data\\211596498612667-0.pdf")

In [76]:
print(len(pdf.pages))
print(pdf.pages[0].extract_text())
print(pdf.pages[1].extract_text())
print(pdf.pages[2].extract_text())
print(pdf.pages[3].extract_text())

30
Sunday, June 20, 2021
Fiche d'inscription 
Nom et prénom d'élève  Blatt Luce
Date de naissance 10 19 1960
Adresse Rue des vincennes, 9
Toulouse, 31500
E-mail marieluceblatt@gmail.com
Téléphone (0033) 607-103468
Cours:      
Horaire:   
Lundi      12h15 Barre à terre   
Lundi Heure
Mardi    9h Barre à terre   
Mardi Heure
          
Jeudi    9h30 pbt     
Jeudi Heure
Vendredi  10h classique moyen 
Vendredi Heure
      
            
Téléverser le Certi cat Médical
CamScanner 06-20-2021 20.39.pdf
pdf
1
Create your own automated PDFs with Jotform PDF Editor- It’s free
Téléverser le Certi cat d’assurance 
extra-scolaire ou assurance civil
66_CamScanner 06-20-2021 20.39_8995.pdf
pdf
Le paiement du cours sera effectué avec Par 4 cheques
(1-10)
 chèques deXxx  €, au total de     + 30€
(valeur de chaque chèque)
adhésion .
Moins 10€ pour le paiement comptant ou trimestre.
 Découvrez notre planning et nos tarifs a
attitudecorpsetdanses.com/tarifs-et-planning
Pour l’abonnement annuel à Attitude

The inscription form provides lots of information about each student. I'll create lists to store these informations, according to the fields filled.

In [190]:
name=[]
birthday=[]
address = []
mail =[]
telephone =[]
legal_representative =[]
course=[]
schedule =[]
course2 =[]
schedule2 =[]
course3=[]
schedule3 =[]
registration =[]
installments =[]
total =[]

In [187]:
def read_pdfs (pdf):
    """open all files and extract all the text information in each
    return info"""
    info = []
    for page in range(len(pdf.pages)):
        info.append(pdf.pages[page].extract_text())
    info = "".join(info).replace("Create your own automated PDFs with Jotform PDF Editor- It’s free","").split("Fiche d'inscription")
    for student in range(1,len(info)):
        nom = info[student].split("Nom et prénom d'élève")[1].split("Date de naissance")[0].strip()
        naissance = info[student].split("Date de naissance")[1].split("Adresse")[0].strip()
        adresse = info[student].split("Adresse")[1].split("E-mail")[0].replace("\n"," ").strip()
        email = info[student].split("E-mail")[1].split("Téléphone")[0].strip()
        try:
            representant_legal = info[student].split("eprésentant légal de l’inscrit (pour")[1].split("les mineurs")[0].strip()
        except:
            representant_legal = 0
        try:
            tel = info[student].split("Téléphone")[1].split("Représentant légal de l’inscrit")[0].strip()
        except:
            tel = info[student].split("Téléphone")[1].split("Cours")[0].strip()
        cours = info[student].split("Cours:")[1].split("Horaire:")[0].replace("\nCours", "").strip()
        try:
            horaire = info[student].split("Horaire:")[1].split("Cours 2")[0].replace("\xa0", "").replace("Heure", "").strip()
            cours2 = info[student].split("Cours 2:")[1].split("Horaire:")[0].replace("\nCours", "").strip()
            try:
                horaire2 = info[student].split("Horaire:")[2].split("Cours 3")[0].replace("\xa0", "").replace("Heure", "").strip()
                cours3 = info[student].split("Cours 3:")[1].split("Horaire:")[0].replace("\nCours", "").strip()
                horaire3 = info[student].split("Horaire:")[3].split("Téléverser")[0].replace("Heure", "").strip()
            except: 
                horaire2= info[student].split("Horaire:")[2].split("Téléverser")[0].replace("\xa0", "").replace("Heure", "").strip()
                cours3 =0
                horaire3=0
        except:
            horaire = info[student].split("Horaire:")[1].split("Téléverser")[0].replace("\xa0", "").replace("Heure", "").strip()
            cours2 =0
            horaire2=0
            cours3 =0
            horaire3=0
        try:
            adhesion = info[student].split("\xa0\xa0\xa0\xa0\xa0+")[1].split("\n(valeur de chaque chèque)")[0].strip()
        except: 
            info[student].split("Please Select")[1].split("adhésion")[0].strip() 
        try:
            paiement_fractionne =info[student].split("Par")[1].split(" cheques")[0].strip()
        except:
            paiement_fractionne =info[student].split("avec")[1].split("chèques")[0].strip()
        paiement_total = info[student].split("au total de")[1].split("€")[0].strip()
        name.append(nom)
        birthday.append(naissance)
        address.append(adresse)
        mail.append(email)
        telephone.append(tel)
        legal_representative.append(representant_legal)
        course.append(cours)
        schedule.append(horaire)
        course2.append(cours2)
        schedule2.append(horaire2)
        course3.append(cours3)
        schedule3.append(horaire3)
        registration.append(adhesion)
        installments.append(paiement_fractionne)
        total.append(paiement_total)        

In [205]:
attitude = pd.DataFrame(zip(name,birthday, address, mail, telephone, legal_representative, course, schedule, course2, schedule2, course3, schedule3, registration, installments, total))
cols = ["Nom et prénom d'élève", "Date de naissance", "Adresse", "E-mail", "Téléphone", "Représentant légal de l’inscrit (pour les mineurs)", "Cours", "Horaire",  "Cours 2", "Horaire 2",  "Cours 3", "Horaire 3", "Adhésion", "Paiement fractionné", "Paiement Total"]

In [206]:
attitude.columns = cols

In [207]:
attitude

,Nom et prénom d'élève,Date de naissance,Adresse,E-mail,Téléphone,Représentant légal de l’inscrit (pour les mineurs),Cours,Horaire,Cours 2,Horaire 2,Cours 3,Horaire 3,Adhésion,Paiement fractionné,Paiement Total
0,Blatt Luce,10 19 1960,"Rue des vincennes, 9 Toulouse, 31500",marieluceblatt@gmail.com,(,0,,Lundi 12h15 Barre à terre\nLundi \nMardi 9h Ba...,0,0,0,0,30€,4,+ 30
1,Paulon Lily,24/07/14,"Renée Aspe, 2 Toulouse, 31000",do_julia@hotmail.com,6,Paulon Julia,Classique,Lundi 17h\nLundi \n\n\n\n\n\n\n1,0,0,0,0,30€,4,500
2,MEZARD Emmanuelle,02/03/1970,"44, rue Sarah Bernhardt, 44, rue Sarah Bernhar...",emmanuelle.mezard@free.fr,),0,BARRE MOYEN,Mardi 19H\nMardi \n\n\n\n\n\n1,PBT + BALLET FITNESS,Jeudi 9H30\nJeudi,BARRE MOYEN ( + pointes ),Vendredi 19h -20h30 \nVendredi,30€,3,880
3,CORBIERE BEATRIX,20/09/1960,"Jean GAYRAL, 83 TOULOUSE, 31200",lacabiche@free.fr,,0,,Lundi 20h30\nLundi \n\n\n\n\n\n\n1,,19h,0,0,30€,1,720
4,François Eve,08/11/1948,"Avenue Winston Churchill, 10 Toulouse, 31100",francoiseve9@gmail.com,0,0,Barre au sol,Lundi 12h15\nLundi \n\n\n\n\n\n\n1,Barre,Vendredi 19h\nVendredi,0,0,30€,3,720
5,BERNIES-ABELANET BRIGITTE,04/04/1958,"avenue Louis Pasteur, 7 SAINTE-FOY-D'AIGREFEUI...",bbbilou@sfr.fr,8,0,,Lundi 12h15 - 13h15\nLundi \nMardi 18h00 - 19h...,,Mercredi 09h00 - 10h00\nMercredi,0,0,30€,1,1000
6,GAIDDON Emma,14/02/2013,"88 avenue saint exupery Toulouse, 31400", orence@gaiddon.com,9,LAPARLIERE Florence,Classique 1,Mercredi 14H15\nMercredi \n\n\n\n\n1,PBT,Mercredi 15H30\nMercredi,0,0,30€,10,720
7,AUSTRUY Pandore,25/06/2014,"30 Rue SAINT LUC, 30 Rue SAINT LUC TOULOUSE, 3...",juliedischer@hotmail.fr,-,AUSTRUY Julien,préparatoire,Lundi 17h\nLundi,0,0,0,0,30€,1,500
8,AUSTRUY Simone,11/12/2017,"30 Rue SAINT LUC, 30 Rue SAINT LUC TOULOUSE, 3...",juliedischer@hotmail.fr,5,AUSTRUY Julien,éveil,Mardi 17h\nMardi,0,0,0,0,30€,1,500
9,Guez Anne Valerie,19/11/1970,"Rue d’Alsace Lorrainr, 44 Toulouse, 31000",gobati@hotmail.fr,2,Guez Anne Valerie,,Mardi 18h\nMardi \n\n\n\n\n\n1,,Vendredi 19h\nVendredi,0,0,30€,3,720
